# 🔄 Churn Prediction Model Eğitimi

Bu notebook, **LightGBM Classifier** kullanarak müşteri kayıp (churn) tahmini için optimize edilmiş bir model eğitmektedir.

## 📋 Model Özellikleri

- **Model Tipi**: LightGBM (Light Gradient Boosting Machine) Classifier
- **Doğrulama**: 50-Fold Stratified Cross Validation
- **Kullanım Amacı**: Müşteri kayıp tahmini ve önleme stratejileri
- **Avantajlar**: XGBoost'a göre daha hızlı eğitim, daha düşük bellek kullanımı


In [ ]:
import pandas as pd
import numpy as np
import joblib
import time
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, classification_report
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("CHURN PREDICTION MODEL EĞİTİMİ")
print("LightGBM Classifier - 50-Fold Stratified Cross Validation")
print("=" * 80)
print()


## 📂 2. Veri Yükleme

Bu adımda, banka müşteri churn veri setini yüklüyoruz.

**Veri Seti**: `datasets/bank_customer_churn_data/Customer-Churn-Records.csv`
- Toplam kayıt sayısını ve sütun sayısını kontrol ediyoruz


In [ ]:
# Veri dosyasını yükle
INPUT_FILE = 'datasets/bank_customer_churn_data/Customer-Churn-Records.csv'
df = pd.read_csv(INPUT_FILE)

print(f"Toplam kayıt: {len(df)}")
print(f"Sütun sayısı: {len(df.columns)}")
print(f"\nİlk 5 satır:")
df.head()


## 🔧 3. Özellik Mühendisliği (Feature Engineering)

Bu bölümde, model performansını artırmak için yeni özellikler türetiyoruz:

### Türetilmiş Özellikler:
- **Balance_per_Product**: Bakiye / Ürün sayısı
  - Müşterinin her ürün başına ne kadar bakiye tuttuğunu gösterir
  
- **Age_Group**: Yaş grubu kategorisi (Young, Adult, Middle, Senior)
  - Yaş aralıklarına göre müşterileri gruplandırır
  
- **Credit_Score_Age_Ratio**: Kredi skoru / Yaş oranı
  - Müşterinin yaşına göre kredi skorunun ne kadar iyi olduğunu gösterir
  
- **Is_High_Value_Active**: Yüksek değerli aktif müşteri (binary)
  - Hem aktif hem de yüksek bakiyeli müşterileri işaretler


In [ ]:
# Türetilmiş özellikler
# 1. Balance_per_Product: Ürün başına bakiye
df['Balance_per_Product'] = df['Balance'] / (df['NumOfProducts'] + 0.1)

# 2. Age_Group: Yaş grubu kategorisi
df['Age_Group'] = pd.cut(df['Age'], bins=[0, 30, 45, 60, 100], labels=['Young', 'Adult', 'Middle', 'Senior'])

# 3. Credit_Score_Age_Ratio: Kredi skoru / Yaş oranı
df['Credit_Score_Age_Ratio'] = df['CreditScore'] / (df['Age'] + 1)

# 4. Is_High_Value_Active: Yüksek değerli aktif müşteri
df['Is_High_Value_Active'] = ((df['IsActiveMember'] == 1) & (df['Balance'] > df['Balance'].mean())).astype(int)

# Hedef ve özellikler
X = df.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
y = df['Exited']

print(f"Türetilmiş özellikler:")
print(f"  - Balance_per_Product: Ürün başına bakiye")
print(f"  - Age_Group: Yaş grubu (Young, Adult, Middle, Senior)")
print(f"  - Credit_Score_Age_Ratio: Kredi skoru / Yaş oranı")
print(f"  - Is_High_Value_Active: Yüksek değerli aktif müşteri (binary)")
print(f"\nToplam özellik sayısı: {X.shape[1]}")
print(f"\nHedef değişken dağılımı:")
print(f"  0 (Churn Yok): {(y == 0).sum()} ({(y == 0).sum() / len(y) * 100:.2f}%)")
print(f"  1 (Churn Var): {(y == 1).sum()} ({(y == 1).sum() / len(y) * 100:.2f}%)")


## 🧹 4. Veri Ön İşleme (Preprocessing)

Bu bölümde, sayısal ve kategorik değişkenleri ayırıp uygun dönüşümler uyguluyoruz.

### Sayısal Değişkenler:
- **StandardScaler** ile ölçeklendirilir (mean=0, std=1)
- Gradient boosting algoritmaları ölçeklendirilmiş verilerle daha iyi çalışır

### Kategorik Değişkenler:
- **OneHotEncoder** ile binary (0/1) sütunlara dönüştürülür
- Geography, Gender, Age_Group değişkenleri kodlanır


In [ ]:
# Sayısal ve kategorik değişkenleri ayır
num_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
            'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
            'Balance_per_Product', 'Credit_Score_Age_Ratio', 'Is_High_Value_Active']

cat_cols = ['Geography', 'Gender', 'Age_Group']

# Preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),  # Sayısal değişkenler StandardScaler ile ölçeklendirilir
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)  # Kategorik değişkenler OneHotEncoder ile kodlanır
])

print(f"Sayısal değişkenler: {len(num_cols)} (StandardScaler ile ölçeklendirilecek)")
print(f"Kategorik değişkenler: {len(cat_cols)} (OneHotEncoder ile kodlanacak)")
print(f"\nSayısal değişkenler: {num_cols}")
print(f"Kategorik değişkenler: {cat_cols}")


## ✂️ 5. Veri Bölme (Train-Test Split)

Modelin genelleme yeteneğini test etmek için veriyi eğitim ve test setlerine ayırıyoruz:

- **Train Set**: Model eğitimi için kullanılacak (%80)
- **Test Set**: Model performansını değerlendirmek için kullanılacak (%20)
- **Stratify**: Hedef değişkenin dağılımını her iki sette de korur (sınıf dengesizliği için önemli)


In [ ]:
# Veri bölme
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,  # %20 test seti
    stratify=y  # Sınıf dağılımını koru
)

print(f"Train set: {X_train.shape[0]} kayıt ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} kayıt ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nTrain set hedef dağılımı:")
print(f"  0 (Churn Yok): {(y_train == 0).sum()} ({(y_train == 0).sum() / len(y_train) * 100:.2f}%)")
print(f"  1 (Churn Var): {(y_train == 1).sum()} ({(y_train == 1).sum() / len(y_train) * 100:.2f}%)")
print(f"\nTest set hedef dağılımı:")
print(f"  0 (Churn Yok): {(y_test == 0).sum()} ({(y_test == 0).sum() / len(y_test) * 100:.2f}%)")
print(f"  1 (Churn Var): {(y_test == 1).sum()} ({(y_test == 1).sum() / len(y_test) * 100:.2f}%)")


## 🎯 6. Model Tanımlama

Bu bölümde, LightGBM modelini ve preprocessing pipeline'ını tanımlıyoruz.

### LightGBM Parametreleri:
- **n_estimators**: 100 (ağaç sayısı)
- **learning_rate**: 0.1 (öğrenme hızı)
- **max_depth**: 5 (ağaç derinliği)
- **subsample**: 0.8 (alt örnekleme oranı)
- **colsample_bytree**: 0.8 (sütun alt örnekleme)
- **boosting_type**: 'gbdt' (Gradient Boosting Decision Tree)

### Pipeline:
Preprocessor ve model bir pipeline içinde birleştirilir. Bu sayede:
- Veri ön işleme ve model eğitimi tek bir adımda yapılır
- Cross-validation sırasında her fold'da preprocessing ayrı ayrı uygulanır (data leakage önleme)


In [ ]:
# LightGBM modeli (optimize edilmiş parametreler)
lgbm_model = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    verbosity=-1,
    boosting_type='gbdt'
)

# Pipeline oluştur (preprocessor + model)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', lgbm_model)
])

print("LightGBM parametreleri:")
print(f"  n_estimators: 100")
print(f"  learning_rate: 0.1")
print(f"  max_depth: 5")
print(f"  subsample: 0.8")
print(f"  colsample_bytree: 0.8")
print(f"  boosting_type: gbdt")
print(f"\nPipeline oluşturuldu: Preprocessor → LightGBM Classifier")


## 🔄 7. 50-Fold Stratified Cross Validation

Bu bölümde, modelin performansını **50-Fold Stratified Cross Validation** ile değerlendiriyoruz.

### Cross Validation Nedir?
- Veriyi 50 eşit parçaya böler
- Her parça sırayla test seti olarak kullanılır, diğerleri eğitim seti olur
- 50 farklı model eğitilir ve performansları ölçülür
- Modelin farklı veri alt kümelerinde ne kadar tutarlı olduğunu gösterir

### Stratified CV:
- Her fold'da sınıf dağılımı korunur (churn yok/var oranı aynı kalır)
- Sınıf dengesizliği olan veri setlerinde önemlidir

### Hesaplanan Metrikler:
- **ROC-AUC**: Sınıflandırma kalitesi
- **Accuracy**: Doğru tahmin oranı


In [ ]:
# 50-Fold Stratified Cross Validation
CV_FOLDS = 50
skf = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True)

print(f"50-Fold Stratified Cross Validation başlatılıyor...")
print(f"Bu işlem birkaç dakika sürebilir...\n")

start_time = time.time()

# ROC-AUC ve Accuracy için cross-validation
cv_auc_scores = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring='roc_auc', n_jobs=-1)
cv_acc_scores = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring='accuracy', n_jobs=-1)

elapsed_time = time.time() - start_time

print(f"✅ Cross-Validation tamamlandı! (Süre: {elapsed_time:.2f} saniye)")
print()
print("CROSS-VALIDATION SONUÇLARI (50-Fold):")
print("-" * 80)
print(f"ROC-AUC Ortalama: {cv_auc_scores.mean():.4f} ({cv_auc_scores.mean()*100:.2f}%)")
print(f"ROC-AUC Std Sapma: {cv_auc_scores.std():.4f} ({cv_auc_scores.std()*100:.2f}%)")
print(f"ROC-AUC Min: {cv_auc_scores.min():.4f} ({cv_auc_scores.min()*100:.2f}%)")
print(f"ROC-AUC Max: {cv_auc_scores.max():.4f} ({cv_auc_scores.max()*100:.2f}%)")
print()
print(f"Accuracy Ortalama: {cv_acc_scores.mean():.4f} ({cv_acc_scores.mean()*100:.2f}%)")
print(f"Accuracy Std Sapma: {cv_acc_scores.std():.4f} ({cv_acc_scores.std()*100:.2f}%)")
print(f"Accuracy Min: {cv_acc_scores.min():.4f} ({cv_acc_scores.min()*100:.2f}%)")
print(f"Accuracy Max: {cv_acc_scores.max():.4f} ({cv_acc_scores.max()*100:.2f}%)")


## 🚀 8. Final Model Eğitimi

Cross-validation sonuçları yeterliyse, tüm eğitim seti ile final modeli eğitiyoruz.


In [ ]:
# Final model tüm train seti ile eğitiliyor
print("Final model tüm train seti ile eğitiliyor...")
pipeline.fit(X_train, y_train)
print("✅ Eğitim tamamlandı!")


## 📊 9. Model Değerlendirme

Final modelin performansını test seti üzerinde değerlendiriyoruz.

### Kullanılan Metrikler:
1. **Accuracy**: Doğru tahmin oranı
2. **ROC-AUC**: Sınıflandırma kalitesi (0.5 = rastgele, 1.0 = mükemmel)
3. **Precision**: Pozitif tahminlerin doğruluğu
4. **Recall**: Gerçek pozitiflerin ne kadarını yakaladığımız
5. **F1-Score**: Precision ve Recall'un harmonik ortalaması


In [ ]:
# Test set tahminleri
y_pred = pipeline.predict(X_test)  # Sınıf tahminleri (0 veya 1)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Olasılık tahminleri (0-1 arası)

# Metrikler
test_accuracy = accuracy_score(y_test, y_pred)
test_roc_auc = roc_auc_score(y_test, y_pred_proba)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

print("=" * 80)
print("TEST SET SONUÇLARI:")
print("=" * 80)
print(f"Accuracy:        {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"ROC-AUC:         {test_roc_auc:.4f} ({test_roc_auc*100:.2f}%)")
print(f"Precision:       {test_precision:.4f} ({test_precision*100:.2f}%)")
print(f"Recall:          {test_recall:.4f} ({test_recall*100:.2f}%)")
print(f"F1-Score:        {test_f1:.4f} ({test_f1*100:.2f}%)")
print()

# Classification Report
print("Classification Report (Test Set):")
print(classification_report(y_test, y_pred, target_names=['Churn Yok', 'Churn Var']))


## 💾 10. Model Kaydetme

Eğitilmiş modeli, preprocessor'ı ve ilgili bilgileri bir paket halinde kaydediyoruz.

**Kaydedilen Dosya**: `churn_model_v1.pkl`

**İçerik**:
- Eğitilmiş pipeline (preprocessor + model)
- Preprocessor (veri ön işleme pipeline'ı)
- Cross-validation metrikleri (50-fold)
- Test set performans metrikleri
- Özellik listeleri (sayısal ve kategorik)


In [ ]:
# Model package
model_package = {
    'pipeline': pipeline,
    'preprocessor': preprocessor,
    'cv_auc_mean': cv_auc_scores.mean(),
    'cv_auc_std': cv_auc_scores.std(),
    'cv_acc_mean': cv_acc_scores.mean(),
    'cv_acc_std': cv_acc_scores.std(),
    'test_accuracy': test_accuracy,
    'test_roc_auc': test_roc_auc,
    'num_features': num_cols,
    'cat_features': cat_cols
}

output_file = 'churn_model_v1.pkl'
joblib.dump(model_package, output_file)
print(f"✅ Model başarıyla kaydedildi: {output_file}")
print(f"\nKaydedilen bilgiler:")
print(f"  - Model: LightGBM Classifier")
print(f"  - Pipeline: Preprocessor + Model")
print(f"  - CV ROC-AUC Ortalama: {cv_auc_scores.mean():.4f} (Std: {cv_auc_scores.std():.4f})")
print(f"  - CV Accuracy Ortalama: {cv_acc_scores.mean():.4f} (Std: {cv_acc_scores.std():.4f})")
print(f"  - Test Accuracy: {test_accuracy:.4f}")
print(f"  - Test ROC-AUC: {test_roc_auc:.4f}")

print("\n" + "=" * 80)
print("🎉 EĞİTİM TAMAMLANDI!")
print("=" * 80)
